In [1]:
import sqlite3
import requests
import pandas as pd
from sqlalchemy import create_engine
from unidecode import unidecode
import numpy as np
from fuzzywuzzy import fuzz, process
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from highlight_text import fig_text
from qbstyles import mpl_style
import requests
from bs4 import BeautifulSoup
import re
import time
import os

pd.set_option('display.max_columns', None)

def extract_tables_from_url(url):
    try:
        tables = pd.read_html(url)  # Try Premier League URL first
        if not tables:  # If no table is found
            raise ValueError  # Raise an exception to be caught
    except:  # Catch the exception when no table is found
        url = url.replace('c9', 'c10')  # Change to Championship URL
        url = url.replace('Premier-League', 'Championship')
        tables = pd.read_html(url)  # Try Championship URL
    return tables

def get_team_name(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    title = soup.find('title').get_text()
    if re.match('.*\d.*', title):
        name = title.split('Scores')[0].strip()
        name = name.split()[1:]
        name = ' '.join(name)
    else:
        name = title.split('Scores')[0].strip()
    return name

def lists_to_dfs(lists):
    dfs = []
    for lst in lists:
        # convert the list to a numpy array
        arr = np.array(lst)
        # reshape the numpy array into a two-dimensional array
        reshaped_arr = arr.reshape(-1, arr.shape[-1])
        cols = ['date','time','round','day','venue','result','gf','ga','opponent',
                    'xg','xga','possession','attendance','captain','formation','referee','match report','notes']
        # create a DataFrame from the reshaped data
        df = pd.DataFrame(reshaped_arr, columns=cols)
        dfs.append(df)
    # Concatenate all the dataframes
    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

def prepare_df(df):
    df = df.drop(columns=['time','day','captain','formation','referee','match report', 'notes','attendance'])
    df = df.dropna()
    df['date'] = pd.to_datetime(df['date'])
    df['gf'] = df['gf'].apply(lambda x: int(x.split(' ')[0]) if isinstance(x, str) else x)
    df['ga'] = df['ga'].apply(lambda x: int(x.split(' ')[0]) if isinstance(x, str) else x)
    df['xg'] = df['xg'].astype(float)
    df['xga'] = df['xga'].astype(float)
    df['possession'] = df['possession'].astype(int)
    df['xg_roll'] = df['xg'].rolling(window = 10, min_periods = 10).mean()
    df['xga_roll'] = df['xga'].rolling(window = 10, min_periods = 10).mean()
    return df

def sort_team_df(url):
    response = requests.get(url)
    if response.status_code == 429:
        retry_after = int(response.headers.get('Retry-After', 5))  # Default to 5 seconds if header is missing
        time.sleep(retry_after)
        response = requests.get(url)  # Retry the request

    team_name = get_team_name(url)
    table_extraction = extract_tables_from_url(url)
    time.sleep(5)  # Sleep for 5 seconds
    df = lists_to_dfs(table_extraction)
    df = prepare_df(df)
    return df

def replace_with_fuzzy_match(original, choices, scorer=fuzz.token_sort_ratio):
    new_val, score = process.extractOne(original, choices, scorer=scorer)
    return new_val

def fetch_and_process_fbref_data(url, type=2):
    # Fetch the data
    response = requests.get(url).text.replace('<!--', '').replace('-->', '')
    
    # Parse the tables from the HTML response
    df = pd.read_html(response, header=1)[type]
    
    # Clean up the data
    if type == 2:
        df = df[~df['Player'].isin(['Player'])]
        df['Nation'] = df['Nation'].str.extract('([A-Z]{3})')
        df['Pos'] = df['Pos'].str.split(',').str[0]
        df = df.drop('Matches', axis=1)
        df.fillna(0, inplace=True)
        
        # Reset the index
        df = df.reset_index(drop=True)
    # elif type == 1:
        #not sure if we should actually do anything
    
    return df

# Function to replace special characters in a player name
def replace_special_characters(name):
    if isinstance(name, str):
        return unidecode(name)
    return name

def replace_special_characters(name):
    if isinstance(name, str):
        return unidecode(name)
    return name

def convert_to_years(age_str):
    try:
        # Split the string on hyphen and return the first part as integer
        return int(age_str.split('-')[0])
    except:
        return age_str

def sort_df(df, season):
    df.drop_duplicates(subset='Player', keep='first', inplace=True)
    df = df.loc[:, ~df.columns.duplicated()]
    df['Age'] = df['Age'].apply(convert_to_years)
    df.loc[:, 'Age':] = df.loc[:, 'Age':].astype(float)
    df['Player'] = df['Player'].apply(replace_special_characters)
    df['season_years'] = season
    return df

In [2]:
########################################################################################
# Retrieve and Sort FPL DATA
########################################################################################
import requests
import pandas as pd
import numpy as np
from unidecode import unidecode
from sqlalchemy import create_engine
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Remove all special chars
def replace_special_characters(name):
    if isinstance(name, str):
        return unidecode(name)
    return name

# Fuzzy matching function
def best_match(row):
    best_score = 0
    best_match = ''
    for item in match_list:
        score = fuzz.ratio(row['Player'], item)
        if score > best_score:
            best_score = score
            best_match = item
    return best_match, best_score

# These few names don't give the right players for fpl data
name_mapping = {
    "Benie Traore": "Benie Adama Traore",
    "Alisson Ramses Becker": "Alisson",
    "Antony Matheus dos Santos": "Antony",
    "Matheus Santos Carneiro Da Cunha": "Matheus Cunha",
    "Danilo dos Santos de Oliveira": "Danilo",
    "Ederson Santana de Moraes": "Ederson",
    "Emerson Leite de Souza Junior": "Emerson",
    "Joao Victor Gomes da Silva": "Joao Gomes",
    "Joelinton Cassio Apolinario de Lira": "Joelinton",
    "Joao Pedro Junqueira de Jesus" : "Joao Pedro",
    "Jorge Luiz Frello Filho" : "Jorginho",
    "Diogo Teixeira da Silva": "Diogo Jota",
    "Manuel Benson Hedilazio": "Benson Manuel",
    "Gabriel Martinelli Silva": "Gabriel Martinelli",
    "Norberto Murara Neto": "Neto",
    "Lucas Tolentino Coelho de Lima": "Lucas Paqueta",
    "Rodrigo Hernandez": "Rodri",
    "Jose Malheiro de Sa": "Jose Sa",
    "Bernardo Veiga de Carvalho e Silva": "Bernardo Silva",
    "Thiago Emiliano da Silva": "Thiago Silva",
    "Victor da Silva": "Vitinho",
    "Willian Borges da Silva": "Willian",
    "Gabriel Fernando de Jesus": "Gabriel Jesus",
    "Emiliano Buendia Stati": "Emi Buendia",
    "Alexandre Moreno Lopera": "Alex Moreno",
    "Igor Julio dos Santos de Paulo": "Igor",
    "Matheus Franca": "Matheus Cunha",
    "Matheus Franca de Oliveira": "Matheus Franca",
    "Norberto Bercique Gomes Betuncal": "Beto",
    "Felipe Augusto de Almeida Monteiro": "Felipe",
    "Murillo Santiago Costa dos Santos": "Murillo",
    "Bryan Gil Salvatierra": "Bryan Gil"
}

# Get in FPL player data
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'
r = requests.get(url)
json = r.json()
json.keys()

elements_df = pd.DataFrame(json['elements'])
elements_types_df = pd.DataFrame(json['element_types'])
teams_df = pd.DataFrame(json['teams'])

teams_df = teams_df[['id','name']]
positions = elements_types_df[['id','plural_name','singular_name']].copy()
positions.loc[:, 'pos_short'] = positions.loc[:,'singular_name'].apply(lambda x: 'GK' if x == 'Goalkeeper' 
                                                            else ('DF' if x == 'Defender' 
                                                            else ('MF' if x == 'Midfielder' 
                                                            else ('FW' if x == 'Forward' else x))))

slim_elements_df = elements_df[['second_name','first_name','team','element_type','selected_by_percent','now_cost',
                                    'minutes','transfers_in','transfers_out','value_season','bonus','bps','total_points', 
                                    'expected_goals', 'expected_assists','expected_goal_involvements','expected_goals_conceded',
                                    'expected_goals_per_90','expected_assists_per_90', 'expected_goal_involvements_per_90', 
                                    'expected_goals_conceded_per_90', 'points_per_game', 'value_form', 'value_season',
                                    'goals_scored', 'assists', 'clean_sheets', 'goals_conceded', 'own_goals', 'penalties_saved',
                                    'penalties_missed', 'yellow_cards', 'red_cards', 'saves','id']]

merged_df_fpl = slim_elements_df.merge(positions, left_on='element_type', right_on='id', how='left')
merged_df_fpl = merged_df_fpl.merge(teams_df, left_on='team', right_on='id', how='left')
merged_df_fpl['full_name'] = merged_df_fpl['first_name'] + ' ' + merged_df_fpl['second_name']
merged_df_fpl['first_name'] = merged_df_fpl['first_name'].apply(replace_special_characters)
merged_df_fpl['second_name'] = merged_df_fpl['second_name'].apply(replace_special_characters)
merged_df_fpl['full_name'] = merged_df_fpl['full_name'].apply(replace_special_characters)

# Get the current working directory
current_dir = os.getcwd()
# base_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(current_dir))))
new_directory = os.path.join(current_dir, "data", "databases")

# Create a new SQLite database (or connect to an existing one)
db_name = os.path.join(new_directory, 'fbref_data_players_latest.db')

# Get player data needed to update fpl data player names
# db_name = 'fbref_data_players_latest.db'
table = 'general'
conn = sqlite3.connect(db_name)
player_data = pd.read_sql_query(f'SELECT * FROM {table}', conn)
conn.close()

# Update FPL Data player names with FBREF data player names 
merged_df_fpl['full_name'] = merged_df_fpl['full_name'].replace(name_mapping)
match_list = merged_df_fpl['full_name'].values
player_data['best_match'], player_data['score'] = zip(*player_data.apply(best_match, axis=1))
# name_mapping_ext = df.set_index('best_match')['Player'].to_dict()
name_mapping_ext = player_data.set_index('best_match')['Player'].to_dict()
merged_df_fpl['full_name'] = merged_df_fpl['full_name'].replace(name_mapping_ext)

# Get the current working directory
current_dir = os.getcwd()
# base_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(current_dir))))
new_directory = os.path.join(current_dir, "data", "databases")

# Create a new SQLite database (or connect to an existing one)
db_name = os.path.join(new_directory, 'fpl_data.db')
engine = create_engine(f'sqlite:///{db_name}')

# # Create a database engine and load fpl data
# engine = create_engine('sqlite:///fpl_data.db')
merged_df_fpl.to_sql('fpl_data', engine, if_exists='replace')

merged_df_fpl[merged_df_fpl['full_name'] == 'Mohamed Salah']

,second_name,first_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,transfers_out,value_season,bonus,bps,total_points,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,expected_goals_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,points_per_game,value_form,value_season,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,id_x,id_y,plural_name,singular_name,pos_short,id,name,full_name
426,Salah,Mohamed,11,3,21.6,130,1786,4669639,5415359,12.8,23,501,167,14.30,6.00,20.30,22.02,0.72,0.3,1.02,1.11,8.0,0.2,12.8,15,9,7,18,0,0,2,2,0,0,308,3,Midfielders,Midfielder,MF,11,Liverpool,Mohamed Salah


In [210]:
# merged_df_fpl.to_csv('merged_fpl.csv')

In [3]:
import requests
import time

# Get the current working directory
current_dir = os.getcwd()
# base_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(current_dir))))
new_directory = os.path.join(current_dir, "data", "databases")

# Create a new SQLite database (or connect to an existing one)
# db_name = os.path.join(new_directory, 'fbref_data_players_latest.db')
db_name = os.path.join(new_directory, 'fpl_data.db')

# Get player data needed to update fpl data player names
# db_name = 'fbref_data_players_latest.db'
table = 'fpl_data'
conn = sqlite3.connect(db_name)
player_data_fpl = pd.read_sql_query(f'SELECT * FROM {table}', conn)
conn.close()

player_data_fpl = player_data_fpl.sort_values('total_points', ascending=False)
player_data_fpl = player_data_fpl[player_data_fpl['minutes'] != 0].reset_index(drop=True)
player_ids = player_data_fpl['id_x'].unique()
# player_ids


def get_player_data(player_id):
    player_url = f'https://fantasy.premierleague.com/api/element-summary/{player_id}/'
    response = requests.get(player_url)
    response.raise_for_status()
    return response.json()

player_ids = player_ids
all_players_data = []

for player_id in player_ids:
    try:
        player_data = get_player_data(player_id)
        all_players_data.append(player_data)
        # print(f"Got data for player ID {player_id}")
        time.sleep(1)  # Sleep for 1 second to avoid rate limiting
    except requests.exceptions.HTTPError as err:
        print(f"HTTP error occurred: {err}")  # Handle errors and continue
    except Exception as e:
        print(f"An error occurred: {e}")
        break

# At this point, all_players_data contains the data for all players
# You can now proceed to save this data or process it as needed
print("Data retrieval complete.")

Data retrieval complete.


In [4]:
# Flatten the 'fixtures' list of dictionaries
history_data = [history for item in all_players_data for history in item['history']]

# Create DataFrame
all_players_data_df = pd.DataFrame(history_data)

all_players_data_df.head(10)

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,308,9,7,5,False,2023-08-13T15:30:00Z,1,1,1,76,0,1,0,1,0,0,0,0,0,0,0,15,16.6,20.2,12.0,4.9,1,0.28,0.31,0.59,1.28,125,0,2243674,0,0
1,308,14,3,5,True,2023-08-19T14:00:00Z,3,1,2,85,1,0,0,1,0,0,1,0,0,0,0,15,41.4,33.0,79.0,15.3,1,1.23,0.11,1.34,1.00,125,163064,2739755,211617,48553
2,308,29,15,5,False,2023-08-27T15:30:00Z,1,2,3,90,0,1,0,1,0,0,0,0,0,0,0,19,28.6,24.9,16.0,7.0,1,0.18,0.12,0.30,2.00,125,-229297,2570086,98738,328035
3,308,37,2,8,True,2023-09-03T13:00:00Z,3,0,4,90,1,0,1,0,0,0,0,0,0,0,0,26,45.2,23.1,115.0,18.3,1,0.80,0.07,0.87,0.66,125,-122416,2505698,74612,197028
4,308,50,20,10,False,2023-09-16T11:30:00Z,1,3,5,90,0,2,0,1,0,0,0,0,0,0,2,35,44.2,57.4,19.0,12.1,1,0.24,1.47,1.71,0.63,125,-22644,2540130,117182,139826
5,308,57,19,10,True,2023-09-24T13:00:00Z,3,1,6,90,1,0,0,1,0,0,0,0,0,0,3,36,42.6,28.8,51.0,12.2,1,1.01,0.27,1.28,1.11,125,312437,2897929,341186,28749
6,308,68,18,1,False,2023-09-30T16:30:00Z,2,1,7,73,0,0,0,1,0,0,0,1,0,0,0,3,1.0,27.8,9.0,3.8,1,0.05,0.39,0.44,1.44,125,232578,3169998,272616,40038
7,308,72,5,15,False,2023-10-08T13:00:00Z,2,2,8,90,2,0,0,2,0,0,0,0,0,0,3,43,75.8,13.7,66.0,15.6,1,1.45,0.07,1.52,2.30,126,189634,3379782,328947,139313
8,308,85,9,16,True,2023-10-21T11:30:00Z,2,0,9,90,2,0,1,0,0,0,0,0,0,0,3,40,71.0,34.1,51.0,15.6,1,1.39,0.05,1.44,0.13,127,380449,3788926,404926,24477
9,308,97,16,8,True,2023-10-29T14:00:00Z,3,0,10,90,1,0,1,0,0,0,0,0,0,0,0,19,35.4,32.3,20.0,8.8,1,0.58,0.13,0.71,0.45,128,504474,4320178,521527,17053


In [5]:
pd.set_option('display.max_columns', None)

cols = ['full_name', 'id_x', 'name', 'pos_short', 'now_cost', 'total_points', 'goals_scored', 'assists','clean_sheets']
merged_df_fpl_test = player_data_fpl[cols]
player_names_df = merged_df_fpl_test[['full_name', 'id_x']]
merged_df_fpl_test['now_cost'] = merged_df_fpl_test['now_cost']/10
merged_df_fpl_test.head(20)

/var/folders/vd/ly4k__lj017dwcj6ngy4n4c00000gn/T/ipykernel_14010/3327511782.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df_fpl_test['now_cost'] = merged_df_fpl_test['now_cost']/10


,full_name,id_x,name,pos_short,now_cost,total_points,goals_scored,assists,clean_sheets
0,Mohamed Salah,308,Liverpool,MF,13.0,167,15,9,7
1,Ollie Watkins,60,Aston Villa,FW,8.7,161,13,14,6
2,Bukayo Saka,19,Arsenal,MF,9.1,159,12,8,9
3,Son Heung-min,516,Spurs,MF,9.6,142,12,6,6
4,Erling Haaland,355,Man City,FW,14.4,141,17,6,6
5,Phil Foden,353,Man City,MF,8.1,132,8,7,9
6,Dominic Solanke,85,Bournemouth,FW,6.9,126,14,3,5
7,Cole Palmer,362,Chelsea,MF,5.8,124,10,7,4
8,Julian Alvarez,343,Man City,FW,6.7,123,8,9,10
9,Jarrod Bowen,526,West Ham,MF,7.7,120,11,2,5


In [6]:
current_dir = os.getcwd()
new_directory = os.path.join(current_dir, "data", "databases")
db_name = os.path.join(new_directory, 'fpl_data_weekly.db')
table_name = 'overall_data'
engine = create_engine(f'sqlite:///{db_name}')
merged_df_fpl_test.to_sql(table_name, con=engine, if_exists='replace')

539

In [7]:
cols_hist = ['element','total_points','minutes','goals_scored',
            'assists','clean_sheets', 'goals_conceded','own_goals', 
            'penalties_saved', 'penalties_missed', 'yellow_cards', 
            'red_cards', 'saves', 'bonus', 'bps', 'starts', 
            'expected_goals', 'expected_assists', 'expected_goal_involvements',
            'expected_goals_conceded','selected', 'transfers_in',
            'transfers_out']
history_df = all_players_data_df[cols_hist]
weekly_player_df = history_df.merge(player_names_df, how='left', right_on='id_x', left_on='element')
weekly_player_df


,element,total_points,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,selected,transfers_in,transfers_out,full_name,id_x
0,308,5,76,0,1,0,1,0,0,0,0,0,0,0,15,1,0.28,0.31,0.59,1.28,2243674,0,0,Mohamed Salah,308
1,308,5,85,1,0,0,1,0,0,1,0,0,0,0,15,1,1.23,0.11,1.34,1.00,2739755,211617,48553,Mohamed Salah,308
2,308,5,90,0,1,0,1,0,0,0,0,0,0,0,19,1,0.18,0.12,0.30,2.00,2570086,98738,328035,Mohamed Salah,308
3,308,8,90,1,0,1,0,0,0,0,0,0,0,0,26,1,0.80,0.07,0.87,0.66,2505698,74612,197028,Mohamed Salah,308
4,308,10,90,0,2,0,1,0,0,0,0,0,0,2,35,1,0.24,1.47,1.71,0.63,2540130,117182,139826,Mohamed Salah,308
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12795,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,40530,0,1833,Mason Holgate,255
12796,255,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0.00,0.00,39835,0,808,Mason Holgate,255
12797,255,0,90,0,0,0,5,0,0,0,0,0,0,0,6,1,0.34,0.01,0.35,2.27,39521,0,19,Mason Holgate,255
12798,255,1,90,0,0,0,1,0,0,0,1,0,0,0,8,1,0.00,0.00,0.00,1.88,39598,280,279,Mason Holgate,255


In [8]:
current_dir = os.getcwd()
new_directory = os.path.join(current_dir, "data", "databases")
db_name = os.path.join(new_directory, 'fpl_data_weekly.db')
table_name = 'general_data'
engine = create_engine(f'sqlite:///{db_name}')
weekly_player_df.to_sql(table_name, con=engine, if_exists='replace')

12800

In [9]:
########################################################################################
# #update general names
########################################################################################
# Get the current working directory
current_dir = os.getcwd()
# base_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(current_dir))))
new_directory = os.path.join(current_dir, "data", "databases")

# Create a new SQLite database (or connect to an existing one)
db_name = os.path.join(new_directory, 'fbref_data_players_latest.db')
conn = sqlite3.connect(db_name)
test_df_new = pd.read_sql_query('SELECT * FROM general', conn)
conn.close()


# Merge test_df_new with merged_df_fpl on the 'full_name' column to bring in the pos_short values
test_df_new = test_df_new.merge(merged_df_fpl[['full_name', 'pos_short']], left_on='Player', right_on='full_name', how='left')

# Use fillna to fill NaN values in the new_pos column with the original values from the Pos column
test_df_new['new_pos'] = test_df_new['pos_short'].fillna(test_df_new['Pos'])

# Assign the values in the new_pos column back to the Pos column
test_df_new['Pos'] = test_df_new['new_pos']

# Optionally, drop the new_pos, full_name, and pos_short columns
test_df_new = test_df_new.drop(columns=['new_pos', 'full_name', 'pos_short'])
# test_df_new.drop(columns=['level_0'], inplace=True)

manual_updates = {
    'Gabriel Jesus': 'FW',
    'Gabriel Martinelli' : 'MF',
    'Cole Palmer' : 'MF'
}

for player, pos in manual_updates.items():
    test_df_new.loc[test_df_new['Player'] == player, 'Pos'] = pos

# Get the current working directory
current_dir = os.getcwd()
# base_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(current_dir))))
new_directory = os.path.join(current_dir, "data", "databases")

# Create a new SQLite database (or connect to an existing one)
db_name = os.path.join(new_directory, 'fbref_data_players_latest.db')
engine = create_engine(f'sqlite:///{db_name}')

# # Create a database engine and load fpl data
# engine = create_engine('sqlite:///fbref_data_players_latest.db')
# test_df_new.to_sql('general', engine, if_exists='replace')
test_df_new.to_sql('general', engine, if_exists='replace', index=False)

539